#Otros datos:
https://www.epdata.es/datos/accidente-avion-datos-estadisticas/205?accion=2
muertos_en_accidente_de_avion_desde_1942.csv
numero_de_accidentes_aereos_desde_1942.csv

para info otros accidentes por año
http://www.planecrashinfo.com/database.htm

https://aviation-safety.net/database/




Organización de Aviación Civil Internacional ( OACI ), estadísticas mundiales de aviación civil y estimaciones de personal de la OACI.
https://datos.bancomundial.org/indicator/IS.AIR.DPRT


#Info de todos los vuelos en el mundo en tiempo real
https://www.flightradar24.com/18.07,-55.02/3

#Paises y ciudades
https://www.forosdelweb.com/f86/base-datos-paises-estados-ciudades-del-mundo-1157860/
https://docs.google.com/spreadsheets/d/1x-t5GdfxbeuO5GtfnX4BYbTC6XBRI8HgxUfHIclR8bw/edit#gid=565147435

#info geopy
#https://www.europeanvalley.es/noticias/geolocalizacion-y-mapas-con-python/

info traductor
https://pypi.org/project/translate/

#info web scraping
https://www.youtube.com/watch?v=fOR4bLgR00I
https://www.youtube.com/results?search_query=+soup+%3D+b%28content%2C%22lxml%22%29+++Additionally%2C+a+403+Forbidden
https://www.youtube.com/results?search_query=web+scraping+php+python+
https://www.youtube.com/watch?v=VJzz1ZqJong


Database Format
Date:	 Date of accident,  in the format - January 01, 2001
Time:	 Local time, in 24 hr. format unless otherwise specified
Airline/Op:	 Airline or operator of the aircraft
Flight #:	 Flight number assigned by the aircraft operator
Route:	 Complete or partial route flown prior to the accident
AC Type:	 Aircraft type
Reg:	 ICAO registration of the aircraft
cn / ln:	 Construction or serial number / Line or fuselage number
Aboard:	 Total aboard (passengers / crew)
Fatalities:	 Total fatalities aboard (passengers / crew)
Ground:	 Total killed on the ground
Summary:	 Brief description of the accident and cause if known

In [1]:
import pandas as pd 
from pandasgui import show
import re
from sklearn import preprocessing
from geopy.geocoders import Nominatim

In [2]:
df = pd.read_csv('AccidentesAviones.csv')

In [4]:
#df.info()
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [3]:
# 1.Se verifica 'Unnamed: 0', se borra ya que podemos usar el mismo indice del df
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'],inplace=True)
    
#df['Unnamed: 0'] = df['Unnamed: 0'].astype('Int32')
#len(df['Unnamed: 0'].unique())

In [4]:
# 2. Se tranforma la columna fechas y HORA declarada
# Se corrigen las horas
#from time import  strftime

def convertir_dig_en_hora(elem):
    # Se verifica que los numeros coincidan con formato hora 
    if int(elem[:2]) < 24:
        h = elem[:2]
    else:
        h = "00"
    # Se verifica que los numeros coincidan con formato minutos 
    if int(elem[-2:]) < 60:
        min = elem[-2:]
    else:
        min = "00"
    return h+":"+min
    
def conversion_dataTime(pdSerie):
    hora = []
    for elem in pdSerie:
        if elem == "?":
        # Se asigna las 0000 a cualquier hora desconocida "?"
            hora.append("00:00")
        elif elem.isdigit(): 
        # Se verifica si son digitos que se puedan transformar a hora
            hora.append(convertir_dig_en_hora(elem))      
        else:
        # Si no son puros digitos se verifica si se pueden sacar al menos 4 digitos
            digitos = []
            for digito in list(elem):
                if digito.isdigit():
                    digitos.append(digito)
            if len(digitos) == 4:
                hora.append(convertir_dig_en_hora("".join(digitos)))
            else:
                hora.append("00:00")
    return hora
 
df["hora"] = conversion_dataTime(df['HORA declarada'])
df.insert(0, 'date', df['fecha']+" "+df["hora"])

#LA fecha no tienen problema porque pd.to_datetime las reconce automaticamente
df["date"] = pd.to_datetime(df["date"])

df.drop(columns=['HORA declarada',"fecha","hora"],inplace=True)


In [5]:
#Codificamos el Operador
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()


opcion = 1
#Se palantean 2  opciones:
#   1. Se ubica la columna al lado de la columna "date"
#   2. Se codifica con un Id y se crea una tabla IdOperador con valores unicos
#      con el objetivo de crear una DB tipo estrella. Solo se deja en la tabla 
#      la columna IdOperador 
# En ambos casos se crea la columna "Militar" para indicar si el vuelo es civil 
# o asociado a una fuerza militar. 


#Siempre se cambian los desconcodios por "Desconocido"
df['OperadOR'].replace('?',"Desconocido",inplace=True)


val_buscar = ['Air Force','Military', 'Fuerza Aerea', 'Fuerza Area', 'Fuerza Aérea']
if opcion == 1:
    df.insert(1,'Operador', df['OperadOR'])
    df.drop(columns=['OperadOR'],inplace=True)
    #Agregamos una coluna para indicar si el operador es militar 
    df.insert(2,"Militar", df["Operador"].str.contains('|'.join(val_buscar) ,case = False,regex=True).astype('Int32'))
elif opcion == 2:
    #df['IdOperador'] = label_encoder.fit_transform(df['OperadOR']) #Lo mismo pero borracho
    df.insert(1, 'IdOperador', label_encoder.fit_transform(df['OperadOR']))
    #Agregamos una coluna para indicar si el operador es militar 
    df.insert(2,'Militar', df['OperadOR'].str.contains('|'.join(val_buscar) ,case = False,regex=True).astype('Int32'))
    #Se crea la tabla Operadores donde se almacenara informacion relativa a los operadores:
    df.rename(columns = {'OperadOR':'Operador'},inplace=True)
    Operadores =df.loc[:, ('IdOperador', 'Operador','Militar')]
    Operadores.drop_duplicates(inplace=True)
    Operadores.sort_values('IdOperador', ascending =True,inplace=True)
    Operadores.reset_index(inplace=True)
    Operadores.drop(columns=['index'],inplace=True)
    #Se elimina la columna del df
    df.drop(columns=['Operador'],inplace=True)
    df.drop(columns=['Militar'],inplace=True)


In [25]:
#Se crea la tabla Operador_accidentes que sirve para relacionar los operadores 
# con la cantidad de accidentes por operador
df_operador = df [['Operador']]
operadores = list(df_operador['Operador'].unique())
cant_Accidentes = []
for operador in operadores:
    cant_Accidentes.append(df_operador['Operador'].str.count(operador).sum())
cant_Accidentes

d = {'Operador': operadores, 'cant_Accidentes':cant_Accidentes}
dfOperador_accidentes = pd.DataFrame(data=d)
dfOperador_accidentes.to_csv('Operador_accidentes.csv')
#dfOperador_accidentes


In [3]:
df.columns

Index(['Unnamed: 0', 'fecha', 'HORA declarada', 'Ruta', 'OperadOR',
       'flight_no', 'route', 'ac_type', 'registration', 'cn_ln', 'all_aboard',
       'PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos',
       'passenger_fatalities', 'crew_fatalities', 'ground', 'summary'],
      dtype='object')

In [3]:
#Se crea la tabla Operador_accidentes que sirve para relacionar los operadores 
# con la cantidad de accidentes por operador
df_ac_type = df [['ac_type']]
ac_type = list(df_ac_type['ac_type'].unique())
cant_Accidentes = []
for tipo_ac in ac_type:
    cant_Accidentes.append(df_ac_type['ac_type'].str.count(tipo_ac).sum())

d = {'ac_type': ac_type, 'cant_Accidentes':cant_Accidentes}
dfac_type_accidentes = pd.DataFrame(data=d)
dfac_type_accidentes.to_csv('ac_type_accidentes.csv')
#dfOperador_accidentes

error: nothing to repeat at position 0

In [6]:
#Codificamos la columna 'route' que es la ruta del vuelo 

#Se crearan las columnas Ruta_Pais_Origen, Ruta_ciudad_Origen,Ruta_Pais_Destino, Ruta_ciudad_Destino 
df['route'].replace('?',"No_identificado",inplace=True)

def normalizar_ruta(df):
    #Se separan los valores de la columna 'route' que debe contener las ciudades ruta del vuelo
    separadores =['/','-',',']
    s2 = df['route'].str.split(pat='|'.join(separadores),expand=True)

    #Opcion #1 usando geopy (esto se debe modificar para guardar todas las coordenadas Lat y Long)
    #El objetivo es, dado una lista de "ciudades" verificar si son todas del mismo pais, de ser asi
    # devolver un string con la lista de ciudades ordenadas de forma alfabetica indicando al final el pais con el codigo del mismo
    # de no pertenecer al mismo pais devolver una lista de los paises a los que pertenecen esas ciudades
    #  nota de tener un valor ["Training", "Demonstration","Air show","Test flight","No_identificado"] devuelve "No_identificado"

    from IPython.display import clear_output
    from geopy.geocoders import Nominatim
    #import time
    #time.sleep(.5)     #Hay que darle tiempo a que responda sino puede dar error

    geolocator = Nominatim(user_agent = "geoapiExercises")

    def buscar_paises(Pdserie):
        Pdserie = Pdserie.unique()
        descarte = ["Training", "Demonstration","Air show","Test flight","No_identificado"] 
        ciudades = []
        paises = []
        for ciudad in Pdserie:
            if ciudad  and ciudad not in descarte:
                ciudades.append(ciudad.strip())
                try:
                    location = geolocator.geocode(ciudad)
                    location = geolocator.reverse(location.raw['lat']+","+location.raw['lon'])
                    paises.append(location.raw['address']['country_code'])  #['country_code']
                except:
                    pass
        paises = list(set(paises))
        if len(paises) > 1:
            return ",".join(paises), 1
        elif len(paises) == 0:
            return "No_identificado",2
        else: 
            ciudades.sort()
            return ",".join(ciudades)+"-"+paises[0],0
    
    resultado =[]
    JHO = 0
    JHO_max = s2.shape[0]
    for i in range(s2.shape[0]):
        JHO = JHO + 1
        clear_output(wait=True)
        print('Completado: ' + str(round(JHO / JHO_max * 100, 2)) + '%')
        resultado.append(buscar_paises(s2.iloc[i,:]))
        
    Vuelo_internacional = []
    Ruta_vuelo = []
    for i in range(len(resultado)):
        Vuelo_internacional.append(resultado[i][1])
        Ruta_vuelo.append(resultado[i][0])
        
    df.insert(3, 'Ruta_vuelo',Ruta_vuelo )
    df.insert(4, 'Tipo_vuelo_Nac_Int',Vuelo_internacional )
    df['Tipo_vuelo_Nac_Int'] = df['Tipo_vuelo_Nac_Int'].astype('Int32')
    return df

#Para hacer la prueba se toma una muestra del dataset despues se cambia df2 por df
#df2 = df.sample(30)
#df2 = df2.reset_index()
#df2.drop(columns=['index'],inplace=True)

#Se trabaja solo con los accidentes desde el 2000 en adelante para hacer una prueba
df2 = df[df['date'].dt.year > 2000]
df2 = df2.reset_index()
df2.drop(columns=['index'],inplace=True)

df2 =normalizar_ruta(df2)

show(df2)

#Opcion 2, usando un listado de paises y ciudades
# Obtenido de:
# https://www.forosdelweb.com/f86/base-datos-paises-estados-ciudades-del-mundo-1157860/
# https://docs.google.com/spreadsheets/d/1x-t5GdfxbeuO5GtfnX4BYbTC6XBRI8HgxUfHIclR8bw/edit#gid=565147435

#List_Paises = pd.read_csv('datos/PAISES.csv')
#List_ciudades = pd.read_csv('datos/CIUDADES DE PAISES.csv')
#List_ciudades[List_ciudades["CIUDAD"] == 'Corsica']


Completado: 100.0%


PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [7]:
#df2.to_csv('Depurado_intermedio.csv')

In [55]:
df2 = pd.read_csv('Depurado_intermedio.csv')


In [ ]:
#Codificamos el lugar del accidente columna 'Ruta'

#df.apply(lambda x: x['Ruta'] == '?', axis=1).sum()
df['Ruta'].replace('?',"No_identificado",inplace=True)

#r = df['Ruta'].str.split(pat=',',expand=True)
#r = df['Ruta'].str.split(pat=',')

#r.shape[1]
#r[r[r.shape[1]-1].notnull()]

In [56]:
if 'Unnamed: 0' in df2.columns:
    df2.drop(columns=['Unnamed: 0'],inplace=True)
    
if 'route' in df2.columns:
    df2.drop(columns=['route'],inplace=True)

df2["date"] = pd.to_datetime(df2["date"])

df2.rename(columns={'Ruta': "Lugar_accidente"},inplace=True)

def cambio_int(Pdserie):
    salida = []
    for elem in Pdserie:
        try:
            salida.append(int(elem))
        except:
            salida.append(0)
    return salida


df2['PASAJEROS A BORDO'].replace('?',0,inplace=True)
df2.rename(columns={'PASAJEROS A BORDO': "passengers"},inplace=True)
df2['passengers'] = cambio_int(df2['passengers'])


df2['crew_aboard'].replace('?',0,inplace=True)
df2['crew_aboard'] = cambio_int(df2['crew_aboard'])

df2.rename(columns={'cantidad de fallecidos': "Total_fatalities"},inplace=True)

df2['passenger_fatalities'].replace('?',0,inplace=True)
df2['passenger_fatalities'] = cambio_int(df2['passenger_fatalities'])

df2['crew_fatalities'].replace('?',0,inplace=True)
df2['crew_fatalities'] = cambio_int(df2['crew_fatalities'])


df2.rename(columns={'ground': "ground_fatalities"},inplace=True)
df2['ground_fatalities'].replace('?',0,inplace=True)
df2['ground_fatalities'] = cambio_int(df2['ground_fatalities'])

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 697 entries, 0 to 696
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  697 non-null    datetime64[ns]
 1   Operador              697 non-null    object        
 2   Militar               697 non-null    int64         
 3   Ruta_vuelo            697 non-null    object        
 4   Tipo_vuelo_Nac_Int    697 non-null    int64         
 5   Lugar_accidente       697 non-null    object        
 6   flight_no             697 non-null    object        
 7   ac_type               697 non-null    object        
 8   registration          697 non-null    object        
 9   cn_ln                 697 non-null    object        
 10  all_aboard            697 non-null    int64         
 11  passengers            697 non-null    int64         
 12  crew_aboard           697 non-null    int64         
 13  Total_fatalities    

In [58]:
show(df2)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [72]:
import re
s1 = df2['summary']

#Info relativa a fallas en despegue
val_buscar = ['on take off','take off','after taking off','taking off','after takeoff','takeoff',]
df2['take_off'] = s1.str.contains('|'.join(val_buscar), flags=re.IGNORECASE, regex=True,na=False).astype('int32')

#Info relativa a problemas en aterrizaje
val_buscar = ['While on approach','During a landing','landing','final approach','attempting to land','land']
df2['landing'] = s1.str.contains('|'.join(val_buscar), flags=re.IGNORECASE, regex=True,na=False).astype('int32')

#Info relativa a secuestros
val_buscar = ['hijacked']
df2['hijacked'] = s1.str.contains('|'.join(val_buscar), flags=re.IGNORECASE, regex=True,na=False).astype('int32')

#Info relativa a secuestros
val_buscar = ['lost contact']
df2['lost_contact'] = s1.str.contains('|'.join(val_buscar), flags=re.IGNORECASE, regex=True,na=False).astype('int32')

In [74]:
df2.to_csv('Depurado01.csv')

#Obtencion de mas informacion

#Procedimiento de : https://www.youtube.com/watch?v=fOR4bLgR00I
Web Scraping - Extracción de datos web con Python 1


In [9]:
import requests
from bs4 import BeautifulSoup as b 
url = "https://aviation-safety.net/statistics/period/stats.php"
#url = "https://www.mercadolibre.com.mx/"
#url = "https://articulo.mercadolibre.com.mx/MLM-1386825775-inversor-de-corriente-coche-12v-a-110v-150w-1-ac-y-2-_JM#reco_item_pos=3&reco_backend=machinalis-homes-pdp-boos&reco_backend_type=function&reco_client=home_second-best-navigation-trend-recommendations&reco_id=fbbd5122-74d9-490e-b15a-d18b3637d43e&c_id=/home/second-best-navigation-trends-recommendations/element&c_element_order=4&c_uid=4d8503ef-e075-49ab-b1e6-f3052d2e6cf4"
#url = "https://articulo.mercadolibre.com.mx/MLM-929342413-maquina-de-pasta-para-batidora-de-soporte-kitchenaid-_JM?variation=87326131747#reco_item_pos=1&reco_backend=machinalis-homes-pdp-boos&reco_backend_type=function&reco_client=home_navigation-trend-recommendations&reco_id=6d8fea94-ae8d-4821-8094-2ebb00ee08d7&c_id=/home/navigation-trends-recommendations/element&c_element_order=2&c_uid=e1a5d46a-ae89-443a-957d-45a3e8e5a156"

html =requests.get(url)
content = html.content 
soup = b(content,"lxml")
print(soup)


ModuleNotFoundError: No module named 'requests'

In [ ]:
titulo1 = soup.find("h1",{"class":"ui-pdp-title"})
print(titulo1.text)
moneda1 = soup.find("span",{"class":"andes-money-amount__currency-symbol"})
precio1 = soup.find("span",{"class":"andes-money-amount__fraction"})
print(moneda1.text,precio1.text)
descuento = soup.find("span",{"class":"ui-pdp-price__second-line__label ui-pdp-color--GREEN ui-pdp-size--MEDIUM"})
print("Descuento %",descuento.text)

In [27]:
#https://trends.google.es/trends/explore?date=all&q=accidente%20aereo
dfgoogleTrends = pd.read_csv('datos/googleTrends_accidenteAereo.csv')
dfgoogleTrends = dfgoogleTrends.reset_index()
d = {'date': dfgoogleTrends['index'][1:], 'Trends': dfgoogleTrends['Categoría: Todas las categorías'][1:]}
df2googleTrends = pd.DataFrame(data=d)
df2googleTrends['date'] =pd.to_datetime(df2googleTrends['date'])
df2googleTrends['Trends'] =pd.to_numeric(df2googleTrends['Trends'])
df2googleTrends.info()
df2googleTrends.to_csv('googleTrends.csv')

In [62]:
#https://datos.bancomundial.org/indicator/IS.AIR.DPRT
dfAPI_IS_AIR = pd.read_csv('datos/API_IS.AIR.csv')
years = dfAPI_IS_AIR.columns[14:-2]
total_vuelos_año = []
for año in years:
    total_vuelos_año.append(dfAPI_IS_AIR[año].sum())


d = {'año': years, 'total_vuelos':total_vuelos_año}
dftotal_vuelos_año = pd.DataFrame(data=d)
dftotal_vuelos_año['año'] = pd.to_datetime(dftotal_vuelos_año['año'])
dftotal_vuelos_año.to_csv('dftotal_vuelos_año.csv')

In [25]:
dfAPI_IS_AIR = pd.read_csv('datos/API_IS.AIR.csv')
Vuelos_Paises = dfAPI_IS_AIR[['Country Name', 'Country Code', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']]
Vuelos_Paises2 = Vuelos_Paises.dropna(subset = ['Country Name']) 
for columna in Vuelos_Paises2.columns:
    if Vuelos_Paises2[columna].isna().sum() >70:
        Vuelos_Paises2.drop(columns=[columna],inplace=True)
Vuelos_Paises2 = Vuelos_Paises2.fillna(0)
Vuelos_Paises2.to_csv('dftotal_vuelos_paises.csv')

C:\Users\JHONATHAN\AppData\Local\Temp\ipykernel_3496\2304080799.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Vuelos_Paises2.drop(columns=[columna],inplace=True)
C:\Users\JHONATHAN\AppData\Local\Temp\ipykernel_3496\2304080799.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Vuelos_Paises2.drop(columns=[columna],inplace=True)
C:\Users\JHONATHAN\AppData\Local\Temp\ipykernel_3496\2304080799.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [26]:
Vuelos_Paises2

,Country Name,Country Code,1999,2000,2001,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,2132.0,2276.0,0.0,0.0000
2,Afganistán,AFG,3400.0,3409.0,0.0,21677.0,25021.000000,17775.000000,21696.0,25920.000000,23532.0,22770.0,24207.0,10454.0,7334.0,4635.7140
4,Angola,AGO,5100.0,4399.0,4354.0,17060.0,12282.000000,13072.000000,14496.0,13716.000000,13116.0,15482.0,13494.0,13978.0,13647.0,3792.0000
5,Albania,ALB,2700.0,3885.0,3974.0,9412.0,10373.210942,10309.179138,11196.0,1992.000000,0.0,306.0,1904.0,2935.0,2558.0,1274.0000
6,Andorra,AND,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000
262,"Yemen, Rep. del",YEM,7800.0,14608.0,14973.0,20600.0,20176.728526,16807.407296,16932.0,16056.000000,5126.0,4538.0,1669.0,2342.0,0.0,560.0000
263,Sudáfrica,ZAF,103800.0,110392.0,122561.0,190441.0,191506.866330,197817.300311,184763.0,199600.522814,209734.0,212865.0,216275.0,234156.0,248747.0,88578.6330
264,Zambia,ZMB,5400.0,6118.0,4886.0,9730.0,11497.000000,8288.000000,7997.0,8050.000000,11130.0,8094.0,9020.0,1241.0,1298.0,777.3744
